In [ ]:
# ==============================
# 📘 Malawi Stock Exchange Analysis
# ==============================

import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1️⃣ Load Data ---
conn = sqlite3.connect("mse_data.db")
df = pd.read_sql("SELECT * FROM daily_data", conn)
conn.close()

print("✅ Data loaded successfully!")
print(df.head())


In [ ]:
# --- 2️⃣ Clean Data ---

df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(['Ticker', 'Date'])

# Convert numeric fields
df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
df['Volume'] = pd.to_numeric(df['Volume'], errors='coerce')

# Drop missing price rows
df.dropna(subset=['Close'], inplace=True)

print("✅ Data cleaned and formatted.")
df.info()


In [ ]:
# --- 3️⃣ Calculate Key Metrics ---

# Daily % Change
df['Daily_Return_%'] = df.groupby('Ticker')['Close'].pct_change() * 100

# 20-day Moving Average
df['MA_20'] = df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(20, min_periods=1).mean())

# Rolling 20-day Volatility (std of daily returns)
df['Volatility_20'] = df.groupby('Ticker')['Daily_Return_%'].transform(lambda x: x.rolling(20, min_periods=1).std())

# Cumulative Return
df['Cumulative_Return'] = df.groupby('Ticker')['Close'].transform(lambda x: x / x.iloc[0] - 1)

# 7-day Volume Trend
df['Vol_MA_7'] = df.groupby('Ticker')['Volume'].transform(lambda x: x.rolling(7, min_periods=1).mean())

print("✅ Metrics calculated successfully!")
df.head()


In [ ]:
# --- 4️⃣ Save Output to Excel ---
df.to_excel("mse_analysis_output.xlsx", index=False)
print("💾 Results saved to 'mse_analysis_output.xlsx'")


In [ ]:
# --- 5️⃣ Visualization: Setup ---
sns.set(style="whitegrid", palette="tab10")

example_ticker = df['Ticker'].unique()[0]
ticker_df = df[df['Ticker'] == example_ticker]

print(f"📊 Example Ticker: {example_ticker}")


In [ ]:
# --- 6️⃣ Plot: Closing Price & Moving Average ---
plt.figure(figsize=(12,6))
plt.plot(ticker_df['Date'], ticker_df['Close'], label="Close Price", linewidth=2)
plt.plot(ticker_df['Date'], ticker_df['MA_20'], label="20-Day MA", linestyle="--")
plt.title(f"{example_ticker} Price Trend (with 20-Day MA)")
plt.xlabel("Date")
plt.ylabel("Price (MWK)")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# --- 7️⃣ Plot: Volatility (20-Day Rolling) ---
plt.figure(figsize=(12,5))
plt.plot(ticker_df['Date'], ticker_df['Volatility_20'], color="orange", label="Volatility (20-Day)")
plt.title(f"{example_ticker} Rolling Volatility")
plt.xlabel("Date")
plt.ylabel("Volatility (%)")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# --- 8️⃣ Plot: Cumulative Returns (All Tickers) ---
plt.figure(figsize=(12,5))
for ticker in df['Ticker'].unique():
    temp = df[df['Ticker'] == ticker]
    plt.plot(temp['Date'], temp['Cumulative_Return'], label=ticker)

plt.title("Cumulative Returns by Ticker")
plt.xlabel("Date")
plt.ylabel("Cumulative Return (%)")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# --- 9️⃣ Summary Performance Table ---
summary = df.groupby('Ticker').agg({
    'Daily_Return_%': 'mean',
    'Volatility_20': 'mean',
    'Cumulative_Return': 'last',
    'Volume': 'mean'
}).rename(columns={
    'Daily_Return_%': 'Avg Daily Return (%)',
    'Volatility_20': 'Avg Volatility (%)',
    'Cumulative_Return': 'Total Return',
    'Volume': 'Avg Volume'
}).reset_index()

print("📋 === Summary Performance ===")
display(summary.round(2))
